In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
 

In [2]:
#import libraries
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping

import numpy as np
import cv2

import PIL.Image as Image
import os

import matplotlib.pylab as plt

import tensorflow as tf
import tensorflow_hub as hub

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense
from keras.layers import Flatten
import tensorflow.keras.backend as K
from keras.models import load_model

from sklearn.linear_model import LogisticRegression
import random

# data reading

In [3]:
#image preproccessing
train = ImageDataGenerator(rescale =1./255)
test= ImageDataGenerator(rescale= 1/255)
valid = ImageDataGenerator(rescale =1./255)

In [4]:
#save data
photo_size = 256
traindata = train.flow_from_directory(directory="/kaggle/input/autistic-children-facial-data-set/train",target_size=(photo_size,photo_size) ,class_mode='binary',
        batch_size=32,
        classes=['non_autistic','autistic'] )
testdata= test.flow_from_directory(directory="/kaggle/input/autistic-children-facial-data-set/test", target_size=(photo_size,photo_size),class_mode='binary',
        batch_size=32,
        classes=['non_autistic','autistic'])
validdata =  valid.flow_from_directory(directory="/kaggle/input/autistic-children-facial-data-set/valid", target_size=(photo_size,photo_size),class_mode='binary',
        batch_size=32,
        classes=['non_autistic','autistic'])

Found 2536 images belonging to 2 classes.
Found 300 images belonging to 2 classes.
Found 100 images belonging to 2 classes.


# mobile  

In [5]:
feature_extractor_model = "https://tfhub.dev/sayannath/mobilevit_xxs_1k_256_fe/1"

pretrained_model_without_top_layer = hub.KerasLayer(feature_extractor_model, input_shape=(photo_size, photo_size, 3), trainable=False)

2023-01-14 22:22:28.481243: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-14 22:22:28.605183: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-14 22:22:28.605955: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-14 22:22:28.608004: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [6]:
model = tf.keras.Sequential([
    pretrained_model_without_top_layer,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation = "relu"),
    tf.keras.layers.Dense(8, activation = "relu"),
    tf.keras.layers.Dense(1, activation = "sigmoid") 
    ])

model.compile(
  optimizer="adam",
  loss=tf.keras.losses.BinaryCrossentropy(),
  metrics=['acc'])

In [7]:
histroy_1 = model.fit(traindata, validation_data = validdata,steps_per_epoch = 50, epochs = 25 )

2023-01-14 22:22:40.100003: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/25


2023-01-14 22:22:51.102736: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


50/50 [==============================] - 26s 200ms/step - loss: 0.6424 - acc: 0.6980 - val_loss: 0.5167 - val_acc: 0.7200
Epoch 2/25
50/50 [==============================] - 7s 147ms/step - loss: 0.4023 - acc: 0.8160 - val_loss: 0.5198 - val_acc: 0.8100
Epoch 3/25
50/50 [==============================] - 6s 119ms/step - loss: 0.3345 - acc: 0.8612 - val_loss: 0.5930 - val_acc: 0.7000
Epoch 4/25
50/50 [==============================] - 6s 119ms/step - loss: 0.2489 - acc: 0.8963 - val_loss: 0.4956 - val_acc: 0.8200
Epoch 5/25
50/50 [==============================] - 6s 127ms/step - loss: 0.2127 - acc: 0.9150 - val_loss: 0.5872 - val_acc: 0.7600
Epoch 6/25
50/50 [==============================] - 6s 110ms/step - loss: 0.1908 - acc: 0.9245 - val_loss: 0.5143 - val_acc: 0.7900
Epoch 7/25
50/50 [==============================] - 6s 112ms/step - loss: 0.1876 - acc: 0.9194 - val_loss: 0.5598 - val_acc: 0.7500
Epoch 8/25
50/50 [==============================] - 6s 119ms/step - loss: 0.1283 - acc

In [8]:
model.evaluate(testdata)

10/10 [==============================] - 2s 164ms/step - loss: 0.7795 - acc: 0.8267


[0.7795234322547913, 0.8266666531562805]

In [9]:
model.evaluate(traindata)

80/80 [==============================] - 8s 100ms/step - loss: 0.0024 - acc: 1.0000


[0.0023597124963998795, 1.0]

In [10]:
model.save("mobile.h5")

# inception

In [11]:
def create_inception_model():
    from tensorflow.keras.applications.inception_v3 import InceptionV3
    base_model = InceptionV3(input_shape = (photo_size, photo_size, 3), include_top = False, weights = 'imagenet')
    for layer in base_model.layers:
        layer.trainable = False
    # for layer_idx in range(len(pretrained_model.layers)):
    #     if layer_idx not in [1,2,3,305,306,307,308,309,310]:
    #         pretrained_model.layers[layer_idx].trainable = False
    from tensorflow.keras.optimizers import RMSprop
    from tensorflow.keras import layers
    x = layers.Flatten()(base_model.output)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.2)(x)

    # Add a final sigmoid layer with 1 node for classification output
    x = layers.Dense(1, activation='sigmoid')(x)
    model = tf.keras.models.Model(base_model.input, x)
    model.compile(optimizer = RMSprop(learning_rate=0.0001),   loss=tf.keras.losses.BinaryCrossentropy(), metrics = ['accuracy'])
    return model
inception_model=create_inception_model()


87924736/87910968 [==============================] - 0s 0us/step


In [12]:
incephist = inception_model.fit(traindata,validation_data=validdata,steps_per_epoch=50, epochs=30)

Epoch 1/30
50/50 [==============================] - 11s 149ms/step - loss: 1.3934 - accuracy: 0.6510 - val_loss: 0.5256 - val_accuracy: 0.6900
Epoch 2/30
50/50 [==============================] - 6s 112ms/step - loss: 0.6093 - accuracy: 0.7189 - val_loss: 0.6176 - val_accuracy: 0.7000
Epoch 3/30
50/50 [==============================] - 6s 118ms/step - loss: 0.5117 - accuracy: 0.7456 - val_loss: 0.5140 - val_accuracy: 0.7400
Epoch 4/30
50/50 [==============================] - 6s 111ms/step - loss: 0.4440 - accuracy: 0.8046 - val_loss: 0.5139 - val_accuracy: 0.7800
Epoch 5/30
50/50 [==============================] - 6s 112ms/step - loss: 0.4271 - accuracy: 0.8169 - val_loss: 0.6325 - val_accuracy: 0.7300
Epoch 6/30
50/50 [==============================] - 6s 115ms/step - loss: 0.3860 - accuracy: 0.8281 - val_loss: 0.4654 - val_accuracy: 0.8100
Epoch 7/30
50/50 [==============================] - 6s 113ms/step - loss: 0.3170 - accuracy: 0.8585 - val_loss: 0.6580 - val_accuracy: 0.7500
Epoch

In [13]:
inception_model.evaluate(traindata)

80/80 [==============================] - 8s 95ms/step - loss: 0.0122 - accuracy: 0.9992


[0.012217245995998383, 0.9992113709449768]

In [14]:
inception_model.evaluate(testdata)

10/10 [==============================] - 2s 140ms/step - loss: 0.9453 - accuracy: 0.8033


[0.9453322291374207, 0.8033333420753479]

In [15]:
inception_model.save("inception.h5")

# vgg

In [16]:
def create_vgg_model():
    # load model without classifier layers
    model = VGG16(include_top=False, input_shape=(photo_size, photo_size, 3))
    """for layer_idx in range(len(model.layers)):
        if layer_idx not in [1,2,3,15,16,17,18]:
            model.layers[layer_idx].trainable = False"""
    # add new classifier layers
    flat1 = Flatten()(model.layers[-1].output)
    class1 = Dense(64, activation='relu')(flat1)
    output = Dense(1, activation='sigmoid')(class1)
    # define new model
    model = Model(inputs=model.inputs, outputs=output)
    return model
vgg_model=create_vgg_model()
vgg_model.compile(optimizer=Adam(learning_rate=0.0001), loss=tf.keras.losses.BinaryCrossentropy()
, metrics=['accuracy'])

58900480/58889256 [==============================] - 0s 0us/step


In [17]:
vgghist=vgg_model.fit(traindata,validation_data=validdata, epochs=30, steps_per_epoch = 40)

Epoch 1/30
40/40 [==============================] - 17s 289ms/step - loss: 0.6764 - accuracy: 0.5953 - val_loss: 0.6461 - val_accuracy: 0.7100
Epoch 2/30
40/40 [==============================] - 11s 271ms/step - loss: 0.5898 - accuracy: 0.6898 - val_loss: 0.7172 - val_accuracy: 0.6200
Epoch 3/30
40/40 [==============================] - 11s 267ms/step - loss: 0.5419 - accuracy: 0.7375 - val_loss: 0.4936 - val_accuracy: 0.7500
Epoch 4/30
40/40 [==============================] - 12s 299ms/step - loss: 0.4783 - accuracy: 0.7707 - val_loss: 0.4682 - val_accuracy: 0.7500
Epoch 5/30
40/40 [==============================] - 11s 262ms/step - loss: 0.4616 - accuracy: 0.7994 - val_loss: 0.4853 - val_accuracy: 0.7500
Epoch 6/30
40/40 [==============================] - 11s 268ms/step - loss: 0.4163 - accuracy: 0.8133 - val_loss: 0.5162 - val_accuracy: 0.7300
Epoch 7/30
40/40 [==============================] - 11s 264ms/step - loss: 0.3816 - accuracy: 0.8264 - val_loss: 0.6088 - val_accuracy: 0.7400

In [18]:
vgg_model.evaluate(traindata)

80/80 [==============================] - 8s 97ms/step - loss: 0.0344 - accuracy: 0.9886


[0.034353096038103104, 0.988564670085907]

In [19]:
vgg_model.evaluate(testdata)

10/10 [==============================] - 3s 281ms/step - loss: 0.7392 - accuracy: 0.8467


[0.7391785383224487, 0.846666693687439]

In [20]:
vgg_model.save("vgg16_model.h5")

# Stacking

In [21]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def acc(y_true, y_pred):
    tp = 0
    total = 0
    
    for i in range(len(y_true)):
        if y_true[i] == y_pred[i]:
            tp+=1
        total+=1
    return (tp / total)*100


dependencies = {'KerasLayer':hub.KerasLayer,
    'acc': f1_m }

In [22]:
import gc
def read_data(list, photo_size = 256):  
    data = []
    for i in range(len(list)):
        for filename in os.listdir(list[i]):
            img  = cv2.imread(list[i] + filename)
            img = cv2.resize(img,(photo_size,photo_size))
            data.append([img, i])
    
    random.shuffle(data)
    x = []
    y = []
    for i in range(len(data)):
        x.append(data[i][0])
        y.append(data[i][1])
    
    x = np.array(x)
    y = np.array(y)
    x = x / 255
    return x, y

train_add_autis = "/kaggle/input/autistic-children-facial-data-set/train/autistic/"
train_add_nonautis = "/kaggle/input/autistic-children-facial-data-set/train/non_autistic/"
x_train, y_train = read_data([train_add_autis, train_add_nonautis])
gc.collect()

3345

In [23]:
all_models = []
model = load_model("/kaggle/working/mobile.h5",custom_objects=dependencies)
all_models.append(model)

model = load_model("/kaggle/working/vgg16_model.h5",custom_objects=dependencies)
all_models.append(model)

model = load_model("/kaggle/working/inception.h5",custom_objects=dependencies)
all_models.append(model)
gc.collect()

3135

In [24]:
stackX = None
for model in all_models:
    gc.collect()
    # make prediction
    yhat = model.predict(x_train, verbose=0)
    # stack predictions into [rows, members, probabilities]
    if stackX is None:
        stackX = yhat #
    else:
        stackX = np.dstack((stackX, yhat))
    print(stackX.shape)
# flatten predictions to [rows, members x probabilities]
    #stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))

2023-01-14 22:39:45.133538: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1994391552 exceeds 10% of free system memory.
2023-01-14 22:39:47.644344: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1994391552 exceeds 10% of free system memory.


(2536, 1)


2023-01-14 22:39:57.606202: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1994391552 exceeds 10% of free system memory.
2023-01-14 22:39:59.799621: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1994391552 exceeds 10% of free system memory.


(2536, 1, 2)


2023-01-14 22:40:13.350232: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1994391552 exceeds 10% of free system memory.


(2536, 1, 3)


In [25]:
stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
gc.collect()

785

In [26]:
def stacked_dataset(members, inputX):
    stackX = None
    for model in members:
        gc.collect()
        # make prediction
        yhat = model.predict(inputX, verbose=0)
        print(yhat.shape)
        # stack predictions into [rows, members, probabilities]
        if stackX is None:
            stackX = yhat #
        else:
            stackX = np.dstack((stackX, yhat))
    # flatten predictions to [rows, members x probabilities]
    stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
    return stackX

In [27]:
# fit a model based on the outputs from the ensemble members
def fit_stacked_model(members, inputX, inputy):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX)
    # fit the meta learner
    model = LogisticRegression() #meta learner
    model.fit(stackedX, inputy)
    return model

In [28]:
model = LogisticRegression() #meta learner
model.fit(stackX, y_train)
gc.collect()

116

In [29]:
model = fit_stacked_model(all_models, x_train,y_train)

(2536, 1)
(2536, 1)
(2536, 1)


In [30]:
# make a prediction with the stacked model
def stacked_prediction(members, model, inputX):
    # create dataset using ensemble
    stackedX = stacked_dataset(members, inputX)
    # make a prediction
    yhat = model.predict(stackedX)
    return yhat

In [31]:
train_add_autis = "/kaggle/input/autistic-children-facial-data-set/test/autistic/"
train_add_nonautis = "/kaggle/input/autistic-children-facial-data-set/test/non_autistic/"


x_test, y_test = read_data([train_add_autis, train_add_nonautis])
gc.collect()

767

In [32]:
yhat = stacked_prediction(all_models, model, x_test)
score = f1_m(y_test/1.0, yhat/1.0)
print(score)
print(acc(y_test/1.0, yhat/1.0))
gc.collect()

(300, 1)
(300, 1)
(300, 1)
tf.Tensor(0.7781817680290939, shape=(), dtype=float64)
79.66666666666666


782